This notebook has embedding size as 32

In [1]:
import numpy as np
import tensorflow as tf

tf.config.run_functions_eagerly(True)
import itertools
import pandas as pd
import math
import random

from sklearn.model_selection import train_test_split

import keras 
from keras.backend import max as MAX
from keras.layers import Activation, Input, concatenate, dot
from keras.layers.core import Dense, Lambda, Reshape
from keras.layers.convolutional import Convolution1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

import nltk
nltk.download('punkt')
from nltk import ngrams
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity

punctuation = list(punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from gensim.models.fasttext import FastText

nltk.download('omw-1.4')

from nltk import ngrams



punctuation = list(punctuation)

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec



#import fasttext as ft
#from fasttext import supervised
from nltk.corpus import stopwords


import nltk





from nltk.stem import WordNetLemmatizer

from nltk import WordPunctTokenizer

nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

#get_ipython().run_line_magic('matplotlib', 'inline')

from nltk import ngrams




punctuation = list(punctuation)

!pip install fasttext
import fasttext


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


You should consider upgrading via the 'c:\users\lenovo\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
#import os
#os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
#gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#for device in gpu_devices:
    #tf.config.experimental.set_memory_growth(device, True)


In [ ]:
#import os
#os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
experiment_mode = 'test'
if(experiment_mode=='train'):
    df = pd.read_csv('C:/Users/Lenovo/Downloads/Project/Data/complaintICD2.csv', encoding = "ISO-8859-1",on_bad_lines='skip')
    df_neg = pd.read_csv('C:/Users/Lenovo/Downloads/Project/Data/NegativeICDDesc.csv', encoding = "ISO-8859-1",on_bad_lines='skip')

    #/content/drive/MyDrive/Full_Data.csv

    #Read the data
    #df=pd.read_csv(data_file,encoding = "ISO-8859-1",on_bad_lines='skip')
    # df = df.drop(df.columns[-3:], axis=1)

    #query with not null values

    df.loc[df['SPED_COMLTEXT'].isnull(),'SPED_COMLTEXT'] = "#"
    df.loc[df['DESCRIPTION'].isnull(),'DESCRIPTION'] = "#"


    #Get unique values of ICDs
    unique_icds = df['SPDD_ICDCODE'].unique()
    print("Number of Unique ICDs are ",len(unique_icds))
    df.shape

Number of Unique ICDs are  11592


In [ ]:
df[df['SPED_COMLTEXT']=='ABD PAIN']
#df.head()

In [ ]:
#df_neg.head()
#df_neg[['SPED_COMLTEXT'=='ABD PAIN'], ['SPDD_ICDCODE']]
df_neg.loc[df_neg['SPED_COMLTEXT']=='ABD PAIN', ['SPED_COMLTEXT', 'SPDD_ICDCODE']]

#df_neg[['SPED_COMLTEXT'==self.X_col],['SPDD_ICDCODE', self.y_col]]

In [4]:
df = df.sort_values(by='SPDD_ICDCODE')

In [5]:
df.shape

(2385894, 7)

In [6]:
df.loc[df['DESCRIPTION'] == ' unspecified"', 'DESCRIPTION'] = '#'

In [7]:
df = df.loc[df['DESCRIPTION'] != '#']


In [8]:
#df.groupby(by='DESCRIPTION').count().sort_values(by='SPDD_ICDCODE')
#df = df[12000:13000]

experiment_mode = 'test'
if(experiment_mode=='test'):
  df = pd.read_csv('C:/Users/Lenovo/Downloads/Project/Data/FullTestConcat.csv', encoding = "ISO-8859-1",on_bad_lines='skip')
  print(df.head())
  print(df.shape)
  #df=df[5000:10000]


In [9]:
df.shape

(2385656, 7)

In [10]:
#Parameters

MAX_QUERY_LENGTH = 10
MAX_DOCUMENT_LENGTH = 10
K = 300 # Dimensionality of the max-pooling layer. See section 3.4.
L = 128 # Dimensionality of latent semantic space. See section 3.5.
FILTER_LENGTH = 3 # Convolution window size.
NEGATIVE_SAMPLE_SIZE = 5
TRIGRAM_INDICES = None
TOTAL_TRIGRAMS = 0


In [11]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        #print("initial token ->")
        #print(tokens)
        tokens = [word for word in tokens if word not in en_stop]
        #print("intermediate tokens->")
        #print(tokens)
        tokens = [word for word in tokens if len(word) > 3]
        #print("final tokens ->")
        #print(tokens)

        preprocessed_text = ' '.join(tokens)
        #print("preprocessed_text ->")
        #print(preprocessed_text)

        return preprocessed_text

In [18]:
embedding_size = 32
#fasttext_model = "C:/Users/Lenovo/Downloads/Project/FT_model/modelFtUnFull.bin"
if(experiment_mode=='train'):
  ft_comp_model = "C:/Users/Lenovo/Downloads/Project/FT_model/FtFullComp32.bin"
# fasttext_model ="C:/Users/Lenovo/Downloads/Project/FT_model/cc.en.300.bin"

if(experiment_mode=='train'):
    s = ""
    text = []

    for i in df['SPED_COMLTEXT']:
        s ="" + i + "." 
        text.append(s)
    #preprocess text is the funcion specified outside the class
    final_corpus = [preprocess_text(sentence) for sentence in text]
    word_punctuation_tokenizer = nltk.WordPunctTokenizer()
    word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in final_corpus]
    
    #embedding_size = 32
    #Total unique words: 53983
    """
    unique_words = set([word for sentence in word_tokenized_corpus for word in sentence])
    total_unique_words = len(unique_words)
    print("Total unique words:", total_unique_words)
    """

    #originally 60
    
    window_size = 4
    min_word = 5
    down_sampling = 1e-2
    print("FastText model about to start")
    ft_model_comp = FastText(word_tokenized_corpus, vector_size = embedding_size, 
                            window=window_size, min_count = min_word, 
                            sample=down_sampling, sg=1, epochs=100)
    #FastText_model = 

    ft_model_comp.save(ft_comp_model)
    
    
ft_model_comp = FastText.load(ft_comp_model)

FastText model about to start


In [19]:
#fasttext_model = "C:/Users/Lenovo/Downloads/Project/FT_model/modelFtUnFull.bin"
if(experiment_mode=='train'):
  ft_desc_model = "C:/Users/Lenovo/Downloads/Project/FT_model/FtFullDesc32.bin"
# fasttext_model ="C:/Users/Lenovo/Downloads/Project/FT_model/cc.en.300.bin"

if(experiment_mode=='train'):
    d = ""
    txt = []

    for i in df['DESCRIPTION']:
        d ="" + i + "." 
        txt.append(d)
    #preprocess text is the funcion specified outside the class
    final_corpus = [preprocess_text(sentence) for sentence in txt]
    word_punctuation_tokenizer = nltk.WordPunctTokenizer()
    word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in final_corpus]
    
    #embedding_size = 32
    #Total unique words: 53983
    #originally 60
    
    window_size = 4
    min_word = 5
    down_sampling = 1e-2
    print("FastText model about to start")
    ft_model_desc = FastText(word_tokenized_corpus, vector_size = embedding_size, 
                            window=window_size, min_count = min_word, 
                            sample=down_sampling, sg=1, epochs=100)
    #FastText_model = 

    ft_model_desc.save(ft_desc_model)
    
    
ft_model_desc = FastText.load(ft_desc_model)

FastText model about to start


In [33]:
word1 = 'radiation'
word2 = 'swelling'
vector1 = ft_model_comp.wv[word1]
vector2 = ft_model_comp.wv[word2]

# Normalize vectors
normalized_vector1 = vector1 / np.linalg.norm(vector1)
normalized_vector2 = vector2 / np.linalg.norm(vector2)

# Calculate cosine similarity
cosine_similarity = np.dot(normalized_vector1, normalized_vector2)

print("Cosine similarity:", cosine_similarity)

Cosine similarity: 0.53490126


In [34]:
df.head(20)

,RCH_PIN,RCH_CVRNO,CREATEDON,SPED_COMLTEXT,SPDD_ICDCODE,DESCRIPTION,SPDD_TYPE
978787,AMC0017937,1380199,2021-01-17,checking,0.0,AJMAN PKG,P
1789542,IBS0420875,1907997,2021-10-03,Pain Rt lower chest -;Nondisplaced Fx of dista...,0.0,AJMAN PKG,F
2215944,IBS0440034,2142731,2022-01-15,Pain & swelling of left ankle,0.0,AJMAN PKG,F
468772,IBS0377205,2255123,2022-03-05,LBA wirh Rt radiation,0.0,AJMAN PKG,F
1273476,SCM0047767,2072803,2021-12-18,laser,0.0,AJMAN PKG,P
1789564,IBS0421197,1884166,2021-09-23,Dysuria,0.0,AJMAN PKG,F
597611,IBS0362029,2338510,2022-04-22,THROAT PAIN BODY PAIN FEVER 1 DAYS N...,0.0,AJMAN PKG,F
1918240,IBS0380174,2253423,2022-03-03,Dysuria pain lower abdomen,0.0,AJMAN PKG,F
468720,IBS0376276,1469675,2021-03-06,reports.,0.0,AJMAN PKG,F
990303,IBS0371812,1034724,2020-06-19,LEAKING,0.0,AJMAN PKG,F


In [ ]:
df_neg.loc[df_neg['SPED_COMLTEXT']=='complaint of pain low back region', ['SPED_COMLTEXT', 'SPDD_ICDCODE', 'DESCRIPTION']]

In [42]:
#embedding_size = int(input("Enter the vector size"))
# Custom data generator : It generates data batch-wise, processes it and sends for training
class ICDRetrivalDataGenerator(tf.keras.utils.Sequence):

    def __init__(self, df, X_col, y_col, mode,
                 batch_size, negative_sample_size, ft_model_comp, ft_model_desc, TRIGRAM_INDICES = None,
                 TOTAL_TRIGRAMS = 0, shuffle=True,
                 single_query='', unique_document_df=None, embedding_size = embedding_size, df_neg=None, neg_unique_document_df=None): #, ft_model = None
        self.df = df.copy()
        self.df_neg = df_neg.copy()
        # complaint text
        self.X_col = X_col
        # description
        self.y_col = y_col
        self.pin_column = 'SPDD_PIN'  # For negative sampling
        self.discarded_words = ['undefined', 'unspecified']  # words to be discarded from queries/documents
        self.replace_characters = [',', ';', '.', '/', ':', '+', '(', ')', '-', '%', '<', '>', '&', '*', '[', ']',
                                    '?', '_']
        self.shuffle = shuffle
        # one out of 'train' or 'predict_on_single'
        self.mode = mode  
        self.batch_size = batch_size
        self.negative_sample_size = negative_sample_size
        self.embedding_size = embedding_size
        self.ft_model_comp = ft_model_comp
        self.ft_model_desc = ft_model_desc
        


        if (self.mode == 'train'):
            #self.create_all_possible_trigrams()
            #self.map_unique_pins_to_icds()
            # this means that unique_document_df actually is a dataframe consisting of the values of both ICD CODES and their DESCRIPTION 
            # since they are mapped to each other
            self.unique_document_df = self.df[['SPDD_ICDCODE', self.y_col]]
            self.unique_document_df = self.unique_document_df.drop_duplicates(subset=['SPDD_ICDCODE'])
            #list of all negative unique documents
            #self.neg_unique_document_df = self.df_neg[['SPED_COMLTEXT'==self.X_col],['SPDD_ICDCODE', self.y_col]]
            self.neg_unique_document_df = df_neg.loc[df_neg['SPED_COMLTEXT']==self.X_col, ['SPDD_ICDCODE', self.y_col]]

            
            #creating fasttext model while training
            
            #FastText_model = 
            
            
        
        if (self.mode == 'predict_on_single'):

            #self.total_letter_trigrams = TOTAL_TRIGRAMS
            #self.trigrams_to_indices = TRIGRAM_INDICES
            self.single_query = single_query
            self.unique_document_df = unique_document_df
            #self.ft_model = ft_model
            #self.fasttext_model = "C:/Users/Lenovo/Downloads/Project/FT_model/modelFtUn.bin"
            #self.ft_model = FastText.load(self.fasttext_model)

    
    # calling fasttxt to generate word vector for a nodel which is called in the constructor itself in training mode
    def fasttext_word_to_vec(self, word, ft_model):
        wordvector = [0] * 32
        # get vector for a word
        #word = 'example'
        try:
            wordvector = ft_model.wv[word]
        except KeyError:
            print("missing n-gram")
            pass
        # returns word vector on the fly
        return wordvector
     
     


    # useful for trigrams
    #removed
    
    # useful for trigrams
    # This converts the input words in text form into vectors
    # removed

    # Function to convert sentence to list of word vectors
    def sentence_to_vectors(self, sentence, sentence_type='query'):
        #print(sentence)
        if (type(sentence) == float):
            sentence = "###"
        # converting sentence to lowercase
        sentence = sentence.lower()
        # splitting sentenc into words by spaces
        words = sentence.split(" ")
        # specifyig the max_length
        max_sentence_length = MAX_QUERY_LENGTH
        if (sentence_type == 'document'):
            # specifying the document length if it's a doc
            max_sentence_length = MAX_DOCUMENT_LENGTH
        # discarding irrelevant words
        words = [x for x in words if x not in self.discarded_words]
        # truncating words to the max limit availabe
        words = words[:max_sentence_length]
        words = words
        # initialisig word vectors
        word_vectors = []
        for word in words:
            if word:
                for specialChar in self.replace_characters:
                  # removinf any special character
                    word = word.replace(specialChar, '')
                for i in range(10):
                    # Replace all digits with ''
                    word = word.replace(str(i), '')
                # calling the fasttext function on a particular word
                if(sentence_type=='document'):
                    vec = self.fasttext_word_to_vec(word, self.ft_model_desc)
                else:
                    vec = self.fasttext_word_to_vec(word, self.ft_model_comp)
                # append word vectors
                word_vectors.append(vec)
            else:
                vec = self.fasttext_word_to_vec('#', ft_model_desc)
                word_vectors.append(vec)
        # padding the sentence if size<max_sentence_length
        if (max_sentence_length > len(word_vectors)):
            word_vectors = word_vectors + [[0.0] * self.embedding_size for x in
                                           range(max_sentence_length - len(word_vectors))]
        # Padding in start and end
        word_vectors = [[0.0] * self.embedding_size] + word_vectors + [
            [0.0] * self.embedding_size] 
        return word_vectors

    def on_epoch_end(self):
        # reshuffling of rows
        self.df = self.df.sample(frac=1)

    def __get_data(self, batches):
        # Generates data containing batch_size samples

        query_vectors = []
        document_vectors = []
        labels = []

        for index, row in batches.iterrows():

            if (self.mode == 'train'):

                #converting query to sentence and then into vector embeddings
                query_vector = self.sentence_to_vectors(row[self.X_col])

                # Add positive sample pair
                query_vectors.append(query_vector)
                #similarly the y values that is the docs
                document_vectors.append(self.sentence_to_vectors(row[self.y_col], sentence_type='document'))
                # labelling them to 1 as in Positive Document
                labels.append(1)
                
                # taking out data from negative table
                negative_document_vectors = self.neg_unique_document_df
                negative_samples_population = negative_document_vectors['DESCRIPTION'].tolist()

                k = self.negative_sample_size
                if len(negative_samples_population) < k:
                    k = len(negative_samples_population)
                if(k!=0):
                  negative_samples = random.sample(negative_document_vectors['DESCRIPTION'].tolist(), k)
                  negative_document_vectors = [self.sentence_to_vectors(sample, sentence_type='document') for sample
                                                  in negative_samples]
                  # Add negative samples pairs
                  query_vectors += [query_vector for x in range(k)]
                  document_vectors += negative_document_vectors
                  # labelling them as 0 coz they are negatve samples
                  labels += [0 for x in range(k)]
                """
                # Extract negative samples
                pin = row[self.pin_column]
                #set of all ICDs which are to be removed
                remove_icds = self.id_idx[pin]
                #no use
                current_icd = row['SPDD_ICDCODE']
                if (current_icd not in remove_icds):
                    remove_icds.append(current_icd)
                
                # print(self.unique_document_df['SPDD_ICDCODE'])
                #removing all the docs correspondong to the remove_ICDs
                available_documents_for_negative_sampling = self.unique_document_df[~self.unique_document_df['SPDD_ICDCODE'].isin(remove_icds)]
                #conveting negative sampl space to list
                available_documents_for_negative_sampling = available_documents_for_negative_sampling[
                    'DESCRIPTION'].tolist()
                #no. of ngative sample size 
                k = self.negative_sample_size
                if (k > len(available_documents_for_negative_sampling)):
                    k = len(available_documents_for_negative_sampling)
                if (k != 0):
                    # negatively selecting from sample space
                    nagative_samples = random.sample(available_documents_for_negative_sampling, k)
                    # conveting them(neg docs) to vectors
                    negative_document_vectors = [self.sentence_to_vectors(sample, sentence_type='document') for sample
                                                 in nagative_samples]
                  
                    # Add negative samples pairs
                    query_vectors += [query_vector for x in range(k)]
                    document_vectors += negative_document_vectors
                    # labelling them as 0 coz they are negatve samples
                    labels += [0 for x in range(k)]
                """

            elif (self.mode == 'predict_on_single'):
                # converting query to vector and appending
                try:
                    query_vectors.append(self.sentence_to_vectors(self.single_query))
                    # converting document  to vector and appending
                    document_vectors.append(self.sentence_to_vectors(row[self.y_col], sentence_type='document'))
                    # labellig them as positive sammples
                    labels.append(1)
                except:
                    print("exception caught")
                    pass

            else:
                raise ("Improper mode")
        # converting query vectors to numpy array
        try:
            query_vectors = np.array(query_vectors)
            # for handling the exception where document_vectors is empty
            #X_batch = [query_vectors, document_vectors]
            # converting document vectors to numpy array
            document_vectors = np.array(document_vectors)
            # converting labels to numpy array
            labels = np.array(labels)
            # x contains query and doc
            X_batch = [query_vectors, document_vectors]
        except:
            pass
        # y here is labels sent to the __getitem__
        return X_batch, labels

    def __getitem__(self, index):
        if (self.mode == 'train'):
            # takes out the batch to be trained
            batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
            #storing the x and y values
            X, y = self.__get_data(batches)
        elif (self.mode == 'predict_on_single'):
            # 
            self.unique_document_names = self.unique_document_df['SPDD_ICDCODE'].values
            # As only unique documents needed for prediction
            batches = self.unique_document_df[index * self.batch_size:(index + 1) * self.batch_size]  
            X, y = self.__get_data(batches)
        else:
            raise ('Improper Mode')
        return X, y

    def __len__(self):
        if (self.mode == 'train'):
            # takes o the no. of batches to train
            return len(self.df) // self.batch_size
        elif (self.mode == 'predict_on_single'):
            # takes out the no. of batches to test
            return math.ceil(len(self.unique_document_df) // self.batch_size) + 1
        else:
            raise ('Improper Mode')

# datagen code removed ->to find out no. of trigrams




In [36]:
# CDSSM Build Model

query = Input(shape=(MAX_QUERY_LENGTH + 2, embedding_size))
doc = Input(shape=(MAX_DOCUMENT_LENGTH + 2, embedding_size))

query_conv = Convolution1D(K, FILTER_LENGTH, padding="valid", input_shape=(MAX_QUERY_LENGTH + 2, embedding_size),
                           activation="tanh", use_bias=False)(query)  # See equation (2).
query_max = Lambda(lambda x: MAX(x, axis=1), output_shape=(K,))(query_conv)
query_sem = Dense(L, activation="tanh", input_dim=K)(query_max)  # See section 3.5.

doc_conv = Convolution1D(K, FILTER_LENGTH, padding="valid", input_shape=(MAX_DOCUMENT_LENGTH + 2, embedding_size),
                         activation="tanh", use_bias=False)(doc)
doc_max = Lambda(lambda x: MAX(x, axis=1), output_shape=(K,))(doc_conv)
doc_sem = Dense(L, activation="tanh", input_dim=K)(doc_max)

cosine_similarities = dot([query_sem, doc_sem], axes=1, normalize=True)  # See equation (4).

probs = cosine_similarities  # See equation (5).  

model = Model(inputs=[query, doc], outputs=probs)
optimizer = keras.optimizers.Adadelta(learning_rate=0.005)
model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics=['accuracy'], run_eagerly=True)
# model.compile(optimizer="adadelta", loss="binary_crossentropy", metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 32)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 12, 32)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 10, 300)      28800       ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 10, 300)      28800       ['input_2[0][0]']                
                                                                                              

In [37]:
# It takes trained model and input dataframe, and computes top-k documents for each query
def fetch_documents_on_queries(model, dataframe, df_neg):
    top_k = 10
    actual_documents = []
    top_predicted_documents = []
    #calling class constructor
    datagen = ICDRetrivalDataGenerator(dataframe,
                                       X_col='SPED_COMLTEXT',
                                       y_col='DESCRIPTION',
                                       mode='train',
                                       batch_size=50,
                                       negative_sample_size=4,
                                       ft_model_comp = ft_model_comp,
                                       ft_model_desc = ft_model_desc,
                                       single_query="", embedding_size = embedding_size, df_neg=df_neg)
    # unique docs
    unique_document_df = datagen.unique_document_df
    # defining the fasTtext model
    #ft_model = datagen.ft_model
    print("Total queries to evaluate ", len(dataframe))
    # 
    for index, row in dataframe.iterrows():
        # stores the complain text
        query = row['SPED_COMLTEXT']
        # 
        datagen_for_query = ICDRetrivalDataGenerator(dataframe,
                                                     X_col='SPED_COMLTEXT',
                                                     y_col='DESCRIPTION',
                                                     mode='predict_on_single',
                                                     batch_size=50,
                                                     negative_sample_size=4,
                                                     ft_model_comp = ft_model_comp,
                                                     ft_model_desc = ft_model_desc,
                                                     TRIGRAM_INDICES = TRIGRAM_INDICES,
                                                     TOTAL_TRIGRAMS = TOTAL_TRIGRAMS,
                                                     single_query=query,
                                                     unique_document_df=unique_document_df,
                                                     embedding_size = embedding_size,
                                                     )
        try:
            prediction_probabilities = model.predict_generator(datagen_for_query, verbose=1)
            prediction_probabilities = [item for sublist in prediction_probabilities for item in sublist]
            predicted_top_document_indices = np.argsort(-np.array(prediction_probabilities), kind='mergesort')[:top_k]
            predicted_documents = [datagen_for_query.unique_document_names[x] for x in predicted_top_document_indices]
            actual_documents.append(row['SPDD_ICDCODE'])
            top_predicted_documents.append(predicted_documents)
            print("Processed ", index)
        except:
            print("Exception caught")
            #tensor = tf.convert_to_tensor(top_predicted_documents)
            #print("Tensor top_predicted_documents shape:", tensor.shape)
            pass
        #tensor = tf.convert_to_tensor(top_predicted_documents)
        #print("Tensor top_predicted_documents shape:", tensor.shape)
    return actual_documents, top_predicted_documents


# This function computes top-k accuracy based on parameter 'k'
def compute_top_k_accuracy(actual_documents, top_predicted_documents, k):
    total = len(actual_documents)
    if(total==0):
        return 0
    correct = 0
    for i in range(total):
        if (actual_documents[i] in top_predicted_documents[i][:k]):
            correct += 1
    accuracy = (correct * 1.0) / total
    return accuracy

In [39]:
if(experiment_mode=='train'):
# ratio of train is to test
    train_test_split_size = 0.8

    # num_samples = df.shape[0]
    # df = df.sample(frac=1).reset_index(drop=True)  # randomly shuffle the complete dataframe
    # split_sample_numbers = int(num_samples * train_test_split_size)

    # train_df = df.iloc[:split_sample_numbers, :]
    # test_df = df.iloc[split_sample_numbers:, :]

    # splitting the train and test model using statify
    train_x, test_x, train_y, test_y = train_test_split(df.iloc[1:,:-1],
                                                        df.iloc[1:,-1],
                                                        stratify=df.iloc[1:,-1],
                                                        train_size=train_test_split_size)
    # train_df x and y
    train_df = pd.concat([train_x, train_y], axis=1)
    # test_df x and y
    test_df = pd.concat([test_x, test_y], axis=1)
    test_df.to_csv("C:/Users/Lenovo/Downloads/Project/Data/TestData32.csv", index=False)
# saving the path of the model where trained data is stored
elif(experiment_mode=='test'):
    test_df = df
save_model_path = "C:/Users/Lenovo/Downloads/Project/Model/model32Full.h5"


In [44]:
if(experiment_mode == 'train'):
    # no. of rows in the training model
    num_samples = train_df.shape[0]
    train_validation_split = 0.8
    split_sample_numbers = int(num_samples * train_validation_split)
    
    # 80% data of the training data use for training rest for validation
    training_df = train_df.iloc[:split_sample_numbers, :]
    validation_df = train_df.iloc[split_sample_numbers:, :]

    # EarlyStopping
    # stops early if the performance doesn't improve.
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=1,
        restore_best_weights=True
    )
    # reducing learing rate when the performance stops improving
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=2,
        verbose=1
    )
    # saves the model weights during training with the given parameters
    model_checkpoint_callback = ModelCheckpoint(
        filepath=save_model_path,
        monitor='val_loss',
        verbose=1,
        mode='min',
        save_best_only=True)

    traingen = ICDRetrivalDataGenerator(training_df, 
                                        X_col='SPED_COMLTEXT',
                                        y_col='DESCRIPTION',
                                        mode='train',
                                        batch_size=120,
                                        negative_sample_size=NEGATIVE_SAMPLE_SIZE, ft_model_comp = ft_model_comp,
                                        ft_model_desc = ft_model_desc,embedding_size = embedding_size, df_neg=df_neg)

    validgen = ICDRetrivalDataGenerator(validation_df,
                                        X_col='SPED_COMLTEXT',
                                        y_col='DESCRIPTION',
                                        mode='train',
                                        batch_size=120,
                                        negative_sample_size=NEGATIVE_SAMPLE_SIZE,ft_model_comp = ft_model_comp,
                                        ft_model_desc = ft_model_desc, embedding_size = embedding_size, df_neg=df_neg)

    history = model.fit_generator(traingen,
                                  validation_data=validgen,
                                  epochs=5,
                                  verbose=1,
                                  class_weight = {0:1,1:5},
                                  callbacks=[early_stopping, reduce_lr, model_checkpoint_callback])

    history.history
    # saving weights of the model
    model.save_weights(save_model_path)
    print(
        "Model Saved")  # Download model as soon as the training finished. You can find the trained model weights on the left folder pannel
    
elif (experiment_mode == 'test'):

    # Load model 
    model.load_weights(save_model_path)
    print("Loaded")
    # reshuffling the rows and adding a new index according to it
    test_df = test_df.sample(frac=1).reset_index(drop=True)

    # test_df = test_df.iloc[:100,:]
    #actual_documents, top_predicted_documents = fetch_documents_on_queries(model, test_df)
    actual_documents, top_predicted_documents = fetch_documents_on_queries(model, test_df.sample(n=1000), df_neg)

    top_3_accuracy = compute_top_k_accuracy(actual_documents, top_predicted_documents, 3)
    print("Top 3 accuracy ", top_3_accuracy)
    top_5_accuracy = compute_top_k_accuracy(actual_documents, top_predicted_documents, 5)
    print("Top 5 accuracy ", top_5_accuracy)


Epoch 1/20
12723/12723 [==============================] - 2266s 178ms/step - loss: 0.1907 - accuracy: 0.9846 - val_loss: 0.0013 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 2/20
12723/12723 [==============================] - 3791s 298ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 5.6144e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 3/20
12723/12723 [==============================] - 2106s 166ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 3.5650e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 4/20
 6830/12723 [===============>..............] - ETA: 15:02 - loss: 0.0016 - accuracy: 1.0000WARNING:tensorflow:Can save best model only with val_loss available, skipping.


12723/12723 [==============================] - 2099s 165ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.5856e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 5/20
12723/12723 [==============================] - 3768s 296ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 2.0373e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 6/20
12723/12723 [==============================] - 2544s 200ms/step - loss: 9.1669e-04 - accuracy: 1.0000 - val_loss: 1.6844e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 7/20
12723/12723 [==============================] - ETA: 0s - loss: 7.6960e-04 - accuracy: 1.0000
Epoch 7: ReduceLROnPlateau reducing learning rate to 4.999999888241291e-06.


12723/12723 [==============================] - 1597s 125ms/step - loss: 7.6960e-04 - accuracy: 1.0000 - val_loss: 1.4364e-04 - val_accuracy: 1.0000 - lr: 0.0050
Epoch 8/20
12723/12723 [==============================] - 1617s 127ms/step - loss: 7.1126e-04 - accuracy: 1.0000 - val_loss: 1.4364e-04 - val_accuracy: 1.0000 - lr: 5.0000e-06
Epoch 9/20
12722/12723 [============================>.] - ETA: 0s - loss: 7.1123e-04 - accuracy: 1.0000
Epoch 9: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-09.
12723/12723 [==============================] - 1681s 132ms/step - loss: 7.1123e-04 - accuracy: 1.0000 - val_loss: 1.4363e-04 - val_accuracy: 1.0000 - lr: 5.0000e-06
Epoch 10/20
12723/12723 [==============================] - 1706s 134ms/step - loss: 7.1126e-04 - accuracy: 1.0000 - val_loss: 1.4364e-04 - val_accuracy: 1.0000 - lr: 5.0000e-09
Epoch 11/20
 3769/12723 [=======>......................] - ETA: 16:57 - loss: 7.1189e-04 - accuracy: 1.0000WARNING:tensorflow:Can save best m

12722/12723 [============================>.] - ETA: 0s - loss: 7.1126e-04 - accuracy: 1.0000
Epoch 11: ReduceLROnPlateau reducing learning rate to 4.999999969612645e-12.
12723/12723 [==============================] - 1641s 129ms/step - loss: 7.1126e-04 - accuracy: 1.0000 - val_loss: 1.4363e-04 - val_accuracy: 1.0000 - lr: 5.0000e-09
Epoch 12/20
12723/12723 [==============================] - 1705s 134ms/step - loss: 7.1125e-04 - accuracy: 1.0000 - val_loss: 1.4363e-04 - val_accuracy: 1.0000 - lr: 5.0000e-12
Epoch 13/20
12723/12723 [==============================] - ETA: 0s - loss: 7.1125e-04 - accuracy: 1.0000
Epoch 13: ReduceLROnPlateau reducing learning rate to 4.999999980020986e-15.
12723/12723 [==============================] - 1686s 132ms/step - loss: 7.1125e-04 - accuracy: 1.0000 - val_loss: 1.4364e-04 - val_accuracy: 1.0000 - lr: 5.0000e-12
Epoch 14/20
 7600/12723 [================>.............] - ETA: 11:14 - loss: 7.1132e-04 - accuracy: 1.0000WARNING:tensorflow:Can save best m

12723/12723 [==============================] - 2268s 178ms/step - loss: 7.1125e-04 - accuracy: 1.0000 - val_loss: 1.4363e-04 - val_accuracy: 1.0000 - lr: 5.0000e-15
Epoch 14: early stopping
Model Saved
